In [1]:
# Cell 1: Fix NumPy Conflict, Install Dependencies & Check GPU
# We force numpy<2.0 to solve the "AttributeError: _ARRAY_API not found"
!pip install -q "numpy<2.0" kaggle tensorflow pandas matplotlib seaborn scikit-learn

import os
import tensorflow as tf
import numpy as np

# Verify the fix
print(f"✅ Fixed NumPy Version: {np.__version__}")
print(f"✅ TensorFlow Version: {tf.__version__}")

# Check if GPU is detected
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"🚀 GPU Detected: {gpu_devices}")
    !nvidia-smi
else:
    print("⚠️ No GPU detected. Check your Runtime settings.")

2025-12-21 14:36:15.848600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-21 14:36:15.985697: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-21 14:36:16.024973: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-21 14:36:16.788264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

✅ Fixed NumPy Version: 1.26.4
✅ TensorFlow Version: 2.10.1
🚀 GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
Sun Dec 21 14:36:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:4B:00.0 Off |                  Off |
| 30%   33C    P8       

In [2]:
# Cell 2: Import Libraries
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

print("✅ Libraries Imported Successfully")

✅ Libraries Imported Successfully


In [5]:
# Cell 3: Download & Unzip (Safe for Remote Servers)
import os

# 1. Use the current folder for Kaggle configuration instead of /root/
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

# 2. Check if kaggle.json was uploaded to the current folder
if os.path.exists('kaggle.json'):
    # Secure the file (Kaggle requires 600 permissions)
    os.chmod('kaggle.json', 0o600)
    print("✅ Kaggle API Token detected and secured.")
    
    # 3. Download Dataset
    print("⬇️ Downloading Dataset...")
    !kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset
    
    # 4. Unzip
    print("📂 Unzipping...")
    !unzip -q brain-tumor-mri-dataset.zip -d brain_tumor_dataset
    print("✅ Download & Unzip Complete.")
    
else:
    print("❌ ERROR: Please upload 'kaggle.json' to this folder first!")

# 5. Verify the "Famous Families"
base_dir = 'brain_tumor_dataset/Training'
if os.path.exists(base_dir):
    classes = os.listdir(base_dir)
    print(f"\nClasses Found: {classes}")
    for c in classes:
        count = len(os.listdir(os.path.join(base_dir, c)))
        print(f" - {c}: {count} images")

✅ Kaggle API Token detected and secured.
⬇️ Downloading Dataset...
Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 66%|█████████████████████████▋             | 98.0M/149M [00:00<00:00, 1.02GB/s]
100%|████████████████████████████████████████| 149M/149M [00:00<00:00, 1.06GB/s]
📂 Unzipping...
✅ Download & Unzip Complete.

Classes Found: ['glioma', 'notumor', 'pituitary', 'meningioma']
 - glioma: 1321 images
 - notumor: 1595 images
 - pituitary: 1457 images
 - meningioma: 1339 images


In [6]:
# Cell 4: Data Generators & Augmentation

IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
TRAIN_DIR = 'brain_tumor_dataset/Training'
TEST_DIR = 'brain_tumor_dataset/Testing'

# 1. Train Generator (With Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize 0-1
    validation_split=0.2,       # 20% for Validation
    rotation_range=15,          # Rotate
    zoom_range=0.1,             # Zoom
    horizontal_flip=True        # Flip
)

# 2. Test Generator (No Augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Load Data
print("--- Loading Training Data (80%) ---")
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    seed=42
)

print("\n--- Loading Validation Data (20%) ---")
val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    seed=42
)

print("\n--- Loading Test Data ---")
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False 
)

--- Loading Training Data (80%) ---
Found 4571 images belonging to 4 classes.

--- Loading Validation Data (20%) ---
Found 1141 images belonging to 4 classes.

--- Loading Test Data ---
Found 1311 images belonging to 4 classes.


In [7]:
# Cell 5: Build ResNet50 Architecture

# 1. Load Base Model (Pre-trained on ImageNet)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 2. Freeze Base Layers
for layer in base_model.layers:
    layer.trainable = False

# 3. Add Custom Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)                          # Prevent overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(4, activation='softmax')(x) # 4 Classes

# 4. Compile
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


2025-12-21 14:39:08.649379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-21 14:39:09.902105: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46698 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:4b:00.0, compute capability: 8.6
2025-12-21 14:39:09.903233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46698 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6


94765736/94765736 [==============================] - 3s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [ ]:
# ==============================================================================
# CELL 6: PROFESSIONAL TWO-STAGE TRAINING
# ==============================================================================

# --- STAGE 1: WARM-UP (Backbone Frozen) ---
print("🚀 Starting Phase 1: Training Custom Head...")
for layer in base_model.layers:
    layer.trainable = False # 

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history_phase1 = model.fit(
    train_generator,
    epochs=5,  # Short warm-up
    validation_data=val_generator
)

# --- STAGE 2: FINE-TUNING (Unfreeze Top Layers) ---
print("\n🔓 Starting Phase 2: Fine-Tuning ResNet50 Blocks...")

# We unfreeze the last 20 layers (Block 5) to adapt to MRI textures
for layer in base_model.layers[-20:]:
    layer.trainable = True # 

# IMPORTANT: Re-compile with a much LOWER learning rate 
model.compile(optimizer=Adam(learning_rate=0.00001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history_phase2 = model.fit(
    train_generator,
    epochs=15, 
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

🚀 Starting Phase 1: Training Custom Head...
Epoch 1/5
143/143 [==============================] - 56s 376ms/step - loss: 1.1556 - accuracy: 0.4734 - val_loss: 1.1747 - val_accuracy: 0.5180
Epoch 2/5
143/143 [==============================] - 54s 378ms/step - loss: 1.0633 - accuracy: 0.5209 - val_loss: 1.1814 - val_accuracy: 0.4198
Epoch 3/5
143/143 [==============================] - 53s 373ms/step - loss: 1.0252 - accuracy: 0.5358 - val_loss: 1.0785 - val_accuracy: 0.6231
Epoch 4/5
143/143 [==============================] - 53s 373ms/step - loss: 0.9963 - accuracy: 0.5541 - val_loss: 1.0519 - val_accuracy: 0.6380
Epoch 5/5
143/143 [==============================] - 54s 375ms/step - loss: 0.9620 - accuracy: 0.5791 - val_loss: 1.0211 - val_accuracy: 0.6380

🔓 Starting Phase 2: Fine-Tuning ResNet50 Blocks...
Epoch 1/15
143/143 [==============================] - 58s 381ms/step - loss: 3.6520 - accuracy: 0.5325 - val_loss: 2.4406 - val_accuracy: 0.3637
Epoch 2/15
143/143 [===================